## Multilinear Engine Algorithm Review

#### Summary
The details of this algorithm have been obtained from the publication *The Multilinear Engine: A Table-Driven, Least Squares Program for Solving Multilinear Problems, including the n-Way Parallel Factor Analysis Model*, url: https://www.jstor.org/stable/1390831.

#### The Pseudocode Algorithm
The ME1 algorithm as described in section 7.1 of the publication.

1. Input control information, structure table, and data. Initialize.
    1. Reset the coefficients for inverse preconditioning of constrained variables:   $c_n = 1.0$.
2. Initialize a sequence of CG steps.
    1. Compute the weights   $w_m$.
    2. Compute the Jacobian. Convert the Jacobian table into the Jacobian matrix for computing the preconditioning coefficients   $p_n = \frac{1}{\sum_{m}w_{m}j_{mn}^2}$.
    3. Set   $\rho = 0$
3. Perform one CG step.The initial solution consists of   $f_n$.
    1. Compute SE factors   $f_n$ (E9.4).
    2. Compute the current fit $y_m$ (E2.1) and the initial object function: $ Q^{(1)} = \sum_{m=1}^{M}w_m(x_m - y_m)^2 $
    3. Compute the components of the Jacobian: $J^{(1)}$, $J^{(2)}$, and $J^{(3)}$ (E4.3, E9.5).
    4. Compute the gradient $g = J^TW(x - y)$ (E9.7).
    5. Compute the transformed gradient **z** by multiplying gradient components with preconditioning coefficients: $z_n = c_{n}p_{n}g_{n}$.
    6. If $\rho = 0$ (if this is the first step of the sequence), set $\beta = 0$, $\rho = g^Tz$, else set $\beta = \frac{g^Tz}{\rho}$, $\rho = g^{T}z$.
    7. Update the accumulated step direction:   $t = \beta{t} + z$
    8. Compute   $\tau = t^{T}J^{T}WJt$,   $\omega = t^{T}J^{T}W(x - y)$. (E9.7)
    9. Compute the initial approximation for the step length, $\alpha = \frac{\omega}{\tau}$
    10. Compute $Q^{(2)} = Q(max(f + \alpha{t}, 1))$ (E9.4, E2.1).
    11. If $Q^{(2)} < Q^{(1)}$, compute nonlinearity-corrected step length as $\alpha = \frac{\alpha}{(2 - (Q^{(1)} - Q^{(2)})\frac{\tau}{\omega^2})}$
    12. else repeatedly try decreasing $\alpha$, computing $Q^{(2)} = Q(max(f = \alpha{t}, 1))$, until $Q^{(2)} < Q^{(1)}$, go to Step 2,
    13. If it was impossible to satisfy $Q^{(2)} < Q^{(1)}$, go to Step 2.
    14. else accept the new solution, set $f = max(f + \alpha{t}, 1)$.
4. Update inverse preconditioning. Denote indexes $n$ of *nonviolating* ($f_n > I_n$) constrained factor elements by $u$, and indexes of violating ($f_n = I_n$) constrained factor elements of $v$.
    1. Increase $c_u$, constrained by $c_u \le 1$.
    2. Decrease $c_v$, constrained by $c_v \ge \epsilon$.
    3. Decrease $t_v$, constrained by $t_v \ge \epsilon$.
5. Write monitoring output, test for convergence.
    1. Output to screen and to log file $Q^{(2)}$ and $\rho$, that is, the goodness-of-fit and the squared length of the preconditioned gradient.
    2. If $|Q^{(1)} - Q^{(2)}| < e$ ($e$ and $k$ are user-specified parameters), and if this condition has been valid in $k$ consecutive steps, assume convergence and go to the end, Step 7.
    3. If the maximum iteration count is exceeded, go to the end, Step 7.
6. Test for continuing this CG sequence.
    1. If convergence is slowing down, and/or the user-specified restart limits allow it, go back to initialiing anothe rCG sequence, Step 2.
    2. Otherwise go back to performing the next CG step in the current sequence, Step 3.
7. The $f_n$ are the solution. Write the results. If the user has requested so, compute and write the approximate Hessian matrix $J^{T}WJ$, the exact Hessian (E8.1), or the weighted Jacobian sparse matrix $W^\frac{1}{2}J$ (for computing these matrices, first convert the Jacobian table into the Jacobian matrix. End the run.



### Variables and Loss Function

#### Variables
These variables are defined in section 2.

$x$: The input data which is of size $m$ features and $n$ samples.<br/>
$m$: The index $m$ enumerates the equations that embody the model to be Solved.<br/>
$x_m$: The data to be fitted, corresponding to a vector containing elements of $x$ which would be denoted as $x_{ijk}$.<br/>
$y_m$: The fitted values, defined by the factor elements.<br/>
$K_m$: The number of product terms in each equation.<br/>
$k$: The index $k$ enumerates the products that are added together in order that their sum be an approximation of $x_m$.<br/>
$f_n$: The vector $f$, consists of values $f_n(n=1, ..., N)$, is the collection of all factor elements of the model.<br/>
$T_{mk}$: The index sets $T_{mk}$ defines one product in one equation.<br/>
$\sigma_m$: The uncertainties $\sigma_m$ are used to generate the weights $w_m$.<br/>

#### Loss Function
As defined in section 2.<br/><br/>
$ Q(x,f) = \sum_{m=1}^{M}{w_m}{e_m^2} = \sum_{m=1}^{M}{w_m}(x_m - y_m)^2$ (2.5)<br/><br/>
$ \hat{f} = arg min_f \sum_{m=1}^{M}w_{m}e_m^2 = arg min_f \sum_{m=1}^{M}(\frac{e_m}{\sigma_m})^2 $ (2.7)<br/><br/>
$ \hat{f} = arg min_f \sum_{m=1}^{M}(\frac{x_m - \sum_{k=1}^{K_m}\prod_{n\in{T_{mk}}}f_n}{\sigma_m})^2$ (2.8)<br/><br/>



### Detailed Algorithm

Taking the pseudo algorithm stated above but including the details necessary for implementation and variable assignment.

#### Terminology

*free factor element*: is used for the unknowns, the profile factors (H) and factor contributions (W).<br/>
*constrained*: indicates such *free factor elements* which are required to be non-negative (H).<br/>
*subexpression*: (SE) variables are defined in terms of the fixed and unknown elements.<br/>

#### Detailed Variables

Defining all variables that are specified in the pseudocode.<br/>
$X$: Input dataset of size $m$ features by $n$ samples.<br/>
$U$: Uncertainty dataset of size $m$ features by $n$ samples.<br/>
$w$: Weights, which are initialially defined from $U$ and feature categories. $w_m = \frac{1}{\sqrt{U_m}}$<br/>
$W$: Diagonal matrix consisting of all weights $w_m$, $m = 1, ..., M$.<br/>
$y$: Fitted values which are the matrix product of $WH$.<br/>
$n$: The number of samples (rows).<br/>
$m$: The number of features (columns).<br/>
$h$: The number of factors.<br/>

$f_n$: Individual components of the loss function. Unsummed equivalent of WH.<br/>
$f_h$: Equivalent of WH, calculated from: $f_n = \sum_{k=1}^{S_h}\prod_{n \in S_{hk}}f_{n} (h = N + 1, ..., N + N^{SE})$.<br/>
$J$: The Jacobian matrix, which is compose of three elements $J^{(1)}$, $J^{(2)}$, and $J^{(3)}$. The Jacobian is calculated from it's components defined $J = \begin{bmatrix}J^{(1)}&J^{(2)}\end{bmatrix} \begin{bmatrix}I\\J^{(3)}\end{bmatrix} $<br/>
$J^{(1)}$: Component of the Jacobian matrix which consists of the partial derivatives of $y_m$ with respect to the free factor elements while keeping the SE factor elements constant.<br/>
$J^{(2)}$: Component of the Jacobian matrix which consists of the partial derivatives of $y_m$ with respect to the SE factor elements.<br/>
$J^{(3)}$: Component of the Jacobian matrix which consists of the partial derivatives $\frac{\partial{f_h}}{\partial{f_n}} (h = N+1, ...,  N+ N^{SE}, n = 1, ..., N)$.<br/>
$I$: Vector which contains the lower limits for factor elements, for non-negatively constrained factor elements $l_n = 0$ and for unconstrained factor elements, $l_n = -\infty$.<br/>
$\epsilon$: Denotes a small constant, such as $10^{-12}$.<br/>

$c$: Coefficients for inverse preconditioning of constrained variables.<br/>
$g$: The computed gradient.<br/>
$p$: The preconditioning coefficients.<br/>
$z$: The transformed gradient $g$ which is the multiplicative product of the inverse preconditioning coefficient, preconditioning coefficient, and the gradient.<br/>
$\rho$: A componet in the calculation of the step direction.<br/> 
$\beta$: A componet in the calculation of the step direction.<br/>
$t$: The step direction.<br/>
$\alpha_k$: The step length.<br/>
$\tau$: A component in the calculation of the step length.<br/>
$\omega$: A component in the calculation of the step length.<br/>
$e$: Convergence change threshold.<br/>
$k$: Convergence change step. When the change in $Q$ over $k$ iterations is less than $e$ the solution is considered converged.<br/>

#### Pseudocode

1. Primary Initialization.
    1. Set $c_n = 1.0$
    2. Input dataset $X$, uncertainty $U$, initialize $W$ and $H$ matrices.
2. CG Step Initialization.
    1. Compute the weights $w_m$.
    2. Compute the Jacobian: $j_{mn} = \frac{\partial{y_m}}{\partial{f_n}} = \sum_{k|n\in T_{mk}} \prod_{l\in T_{mk}n}f_l$. This can be defined as the change in the WH over the change in the weighted/uncertainty squared residuals. The preconditioning coefficients are calculated using the Jacobian: $p_n = \frac{1}{\sum_{m}w_{m}j_{mn}^{2}}$
    3. Set $\rho = 0$.
3. Perform one CG step.
    1. Compute SE factors $f_h = \sum_{k=1}^{S_h}\prod_{n \in S_{nk}}f_n$
    2. Compute current fit $y_m$ and the initial object function: $ Q^{(1)} = \sum_{m=1}^{M}w_m(x_m - y_m)^2 $
    3. <span style="color:red">Compute components of the Jacobian: $j_{mn} = \frac{\partial{y_m}}{\partial{f_n}} = \sum_{k|n\in T_{mk}} \prod_{l\in T_{mk}n}f_l$</span>
    4. Compute the gradient:  $g = J^TW(x - y) = J^{(1)T}W(x - y) + J^{(3)T}(J^{(2)T}W(x - y))$ using the components of the Jacobian.
    5. Compute the transformed gradient $z_n = c_{n}p_{n}g_{n}$.
    6. If $\rho = 0$, first step, set $\beta = 0$, $\rho = g^{T}z$, else set $\beta = \frac{g^{T}z}{\rho}$, $\rho = g^T{z}$.
    7. Update the step direction: $t = \beta t + z$.
    8. Compute $\tau = t^{T}J^{T}WJt$, $\omega = t^{T}J^{T}W(x - y)$
    9. Compute initial approximation for the step length, $\alpha = \frac{\omega}{\tau}$.
    10. Compute $Q^{(2)} = Q(max(f + \alpha t, 1))$. $f$ is calculated as show in step 3.1/A 
    11. If $Q^{(2)} < Q^{(1)}$, compute nonlinearity-corrected step length as: $\alpha = \frac{\alpha}{(2 - (Q^{(1)} - Q^{(2)})\frac{\tau}{\omega^2})}$
    12. Else, repeatedly try decreasing $\alpha$, computing $Q^{(2)} = Q(max(f + \alpha t, 1))$, until $Q^{(2)} < Q^{(1)}$.
    13. If it was impossible to satisfy $Q^{(2)} < Q^{(1)}$, go to Step 2.
4. Update inverse preconditioning. Denote indexes $n$ of *nonviolating* ($f_n > I_n$) constrained factor elements by $u$, and indexes of violating ($f_n = I_n$) constrained factor elements by $v$.
    1. Increase $c_u$, constrained by $c_u \le 1$.
    2. Decrease $c_v$, constrained by $c_v \ge \epsilon$.
    3. Decrease $t_v$, constrained by $t_v \ge \epsilon$.
5. Write monitoring output, test for convergence.
    1. Output to screen and to log file $Q^{(2)}$ and $\rho$, that is, the goodness-of-fit and the squared length of the preconditioned gradient.
    2. If $|Q^{(1)} - Q^{(2)}| < e$ ($e$ and $k$ are user-specified parameters), and if this condition has been valid in $k$ consecutive steps, assume convergence and go to the end, Step 7.
    3. If the maximum iteration count is exceeded, go to the end, Step 7.
6. Test for continuing this CG sequence.
    1. If convergence is slowing down, and/or the user-specified restart limits allow it, go back to initialiing anothe rCG sequence, Step 2.
    2. Otherwise go back to performing the next CG step in the current sequence, Step 3.
7. The $f_n$ are the solution. Write the results. If the user has requested so, compute and write the approximate Hessian matrix $J^{T}WJ$, the exact Hessian (E8.1), or the weighted Jacobian sparse matrix $W^\frac{1}{2}J$ (for computing these matrices, first convert the Jacobian table into the Jacobian matrix. End the run.

In [2]:
import os
import sys
import copy
import logging
import time
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
from src.data.datahandler import DataHandler 

In [3]:
input_file = os.path.join("D:\\", "projects", "nmf_py", "data", "Dataset-BatonRouge-con.csv")
uncertainty_file = os.path.join("D:\\", "projects", "nmf_py", "data", "Dataset-BatonRouge-unc.csv")
output_path = os.path.join("D:\\", "projects", "nmf_py", "output", "BatonRouge")

In [4]:
dh = DataHandler(
        input_path=input_file,
        uncertainty_path=uncertainty_file,
        output_path=output_path,
        index_col='Date'
    )

23-May-23 11:12:31 - Input and output configured successfully


In [9]:
V = dh.input_data_processed
U = dh.uncertainty_data_processed

weights = 1/np.sqrt(U)

m, n = V.shape
n_components = 4
seed = 42
rng = np.random.default_rng(seed)

V_avg = np.sqrt(np.mean(V, axis=0) / n_components)
H = V_avg * rng.standard_normal(size=(n_components, n)).astype(V.dtype, copy=False)
H = np.abs(H)

V_avg2 = np.sqrt(np.mean(V, axis=1) / n_components)
V_avg2 = V_avg2.reshape(len(V_avg2), 1)
W = np.multiply(V_avg2, rng.standard_normal(size=(m, n_components)).astype(V.dtype, copy=False))

In [134]:
# Compute the coefficients for inverse preconditioning
c = np.ones(shape=(m))
c.shape

(307,)

In [135]:
# Computing the Jacobian (Step 2.2)
j = np.square((V - np.matmul(W, H))/U)
j.shape

(307, 41)

In [32]:
# preconditioning coefficients (Step 2.2)
p = []
for i in range(n):
    p_n = 1 / np.sum(weights * j[i]**2, axis=1)    # axis=1 => (41, 307), axis=0 => (41, 41)
    p.append(p_n)
p = np.array(p).T
p.shape

(307, 41)

In [133]:
# Compute SE factors f_h (Step 3.1)
f = []
_h = np.zeros(shape=H.shape)
_w = np.zeros(shape=W.shape)
for i in range(m):
    fi = []
    for k in range(n_components):
        fk = []
        wk = W[i, k]
        _w[i, k] = wk
        for j in range(n):
            hj = H[k, j]
            _h[k, j] = hj
            fk.append(wk *hj)
        fi.append(fk)
    f.append(fi)
f = np.array(_f)
f.shape

(307, 4, 41)

In [34]:
# Compute current fit y_m (Step 3.2)
y = np.matmul(W, H)
Q1 = np.sum(weights * ((V - y)**2))
Q1

1309548.0

In [35]:
# Compute the components of the Jacobian (Step 3.3)


In [132]:
# Compute the gradient (Step 3.4)
mi = 0                     # mi=0->m

g = []
for i in range(m):
    Wi = np.diagflat(weights[i])
    r = (V[i] - y[i])
    r = r.reshape(len(r), 1)
    wr = Wi * r
    gi = np.matmul(j[i], wr)
    g.append(gi)
g = np.array(g)
g

array([[-2.3191492e-04,  1.9877819e+00, -3.3903696e+02, ...,
         2.0886194e-02,  1.6315260e+01,  3.6491001e-01],
       [ 1.2922232e+02,  2.6506693e+02,  3.7786832e+02, ...,
         3.1551428e+02,  5.4215935e+01,  5.1014882e-01],
       [ 6.7611023e+02,  7.8001135e+02,  1.5131453e+03, ...,
         1.5440063e+03,  4.5510075e+01,  5.3064883e-01],
       ...,
       [ 1.0032913e+03,  4.5623413e+02,  1.2805981e+02, ...,
         2.2779692e+02,  4.0964054e+01,  3.4915167e-01],
       [ 8.9038501e+02,  1.1491959e+03,  1.5379944e+02, ...,
         6.5550232e+02,  9.2901901e+01,  4.8890892e-01],
       [ 2.5032963e+01,  1.0410875e+02,  1.9818988e+02, ...,
         8.5854813e+01, -2.2706806e+01,  3.8489667e-01]], dtype=float32)

In [73]:
# Transform gradient (Step 3.5)
_c = c.reshape(len(c), 1)
z = np.multiply(_c, np.multiply(p, g))
z.shape

(307, 41)

In [74]:
# if first step rho = 0 (Step 3.6 option 1)
beta = 0
rho = np.matmul(g.T, z)
rho.shape

(41, 41)

In [75]:
# else (Step 3.6 option 2)
rho2 = rho
beta2 = np.matmul(g.T, z)/rho2
rho2 = np.matmul(g.T, z)
beta2.shape

(41, 41)

In [79]:
# Update the accumulated step direction (Step 3.7)
t0 = np.ones(shape=V.shape)
t = beta * t0 + z
t.shape

(307, 41)

In [96]:
# Compute tau (Step 3.8)
tau = []
for i in range(m):
    wi = np.diagflat(weights[i])
    ti = t[i]
    ji = j[i]
    tau0 = np.multiply(ji, ti)
    tau1 = np.matmul(wi, tau0)
    tau2 = np.multiply(ji.T, tau1)
    _tau = np.multiply(ti.T, tau2)
    tau.append(_tau)
tau = np.array(tau)
tau

array([[1.28755634e-27, 7.16795278e-11, 3.63778089e-08, ...,
        1.04994578e-22, 6.35353560e-16, 3.34495543e-18],
       [3.99184104e-08, 9.35080555e-04, 3.10771073e-08, ...,
        2.43210919e-08, 7.37150575e-14, 8.25843785e-17],
       [1.07842215e-05, 2.29371315e-02, 1.73940801e-06, ...,
        2.20582297e-06, 9.92332512e-14, 5.65592506e-17],
       ...,
       [1.15183197e-05, 3.64476111e-03, 1.35049354e-09, ...,
        1.80147751e-08, 1.24755936e-14, 7.43426273e-18],
       [1.22959701e-05, 7.91438051e-02, 1.26532217e-09, ...,
        2.41504241e-07, 1.58244516e-13, 1.31232166e-17],
       [9.29818019e-11, 5.15299282e-05, 9.61618513e-09, ...,
        5.23400134e-10, 4.98937633e-14, 1.86001696e-17]])

In [101]:
# Compute omega (Step 3.8)
omega = []
for i in range(m):
    wi = np.diagflat(weights[i])
    ti = t[i]
    ji = j[i]
    ri = (V[i] - y[i])
    omega0 = np.matmul(wi, ri)
    omega1 = np.multiply(ji.T, omega0)
    _omega = np.multiply(ti.T, omega1)
    omega.append(_omega)
omega = np.array(omega)
omega

array([[5.03354162e-16, 4.56243365e-06, 2.79573544e-04, ...,
        1.13544109e-12, 3.20615795e-07, 5.80442738e-08],
       [6.00040702e-04, 1.59200330e-01, 7.27560222e-04, ...,
        5.59015485e-04, 1.11770197e-05, 4.29426831e-07],
       [1.10568391e-02, 1.13131020e+00, 5.75572249e-03, ...,
        6.61488084e-03, 4.69600887e-06, 2.90549784e-07],
       ...,
       [8.97126859e-03, 1.86226922e-01, 3.89385108e-05, ...,
        1.34288192e-04, 1.70516355e-06, 4.90984830e-08],
       [1.20407974e-02, 1.76166575e+00, 5.77115798e-05, ...,
        1.12720801e-03, 1.02914317e-05, 1.01500942e-07],
       [1.62863684e-05, 2.54675180e-02, 2.35751490e-04, ...,
        4.74775509e-05, 1.43387941e-06, 1.51962661e-07]])

In [102]:
# Compute initial approximation for the step length (Step 3.9)
alpha = omega / tau
alpha

array([[3.90937582e+11, 6.36504424e+04, 7.68527715e+03, ...,
        1.08142831e+10, 5.04625793e+08, 1.73527795e+10],
       [1.50316783e+04, 1.70253065e+02, 2.34114525e+04, ...,
        2.29848021e+04, 1.51624649e+08, 5.19985545e+09],
       [1.02527930e+03, 4.93222181e+01, 3.30901228e+03, ...,
        2.99882671e+03, 4.73229368e+07, 5.13708687e+09],
       ...,
       [7.78869559e+02, 5.10944110e+01, 2.88328004e+04, ...,
        7.45433631e+03, 1.36679953e+08, 6.60435133e+09],
       [9.79247452e+02, 2.22590479e+01, 4.56101863e+04, ...,
        4.66744601e+03, 6.50349976e+07, 7.73445604e+09],
       [1.75156515e+05, 4.94227702e+02, 2.45161139e+04, ...,
        9.07098562e+04, 2.87386501e+07, 8.16996108e+09]])

In [137]:
# Update f
fh = np.matmul(W, H)
new_f = fh + alpha * t
new_f

array([[-3.11052010e-01,  1.33129102e+00,  1.20492272e+00, ...,
         1.91288744e+00,  1.62632098e+01,  2.76572546e+03],
       [ 3.97332746e-01, -5.53582490e-03, -6.50607721e-01, ...,
        -4.62248521e-01,  3.64149360e+01,  4.37808810e+03],
       [-9.26325394e-01, -2.09863118e+00, -1.60060021e+00, ...,
        -1.64683616e+00,  5.56339440e-01,  2.79892413e+03],
       ...,
       [-8.06893268e-01, -6.73559948e-01, -7.42896794e-02, ...,
        -1.56420443e-01,  2.44207083e+00,  9.21232754e+02],
       [-1.05487798e+00, -2.22889983e+00, -9.52063112e-02, ...,
        -6.08232258e-01, -7.72598865e+00,  1.59024396e+03],
       [ 1.37923379e+00,  1.08720918e+00, -1.55451185e-01, ...,
         3.78035104e-01,  1.54364336e+01,  3.23332299e+03]])

In [142]:
# Issue is that this process is for updating the product and not the composing matrices W and H
# Rewrite the process to target one of the composing matrices then switch

# Keep H constant and update W
import copy


W0 = copy.copy(W)
H0 = copy.copy(H)

X = copy.copy(V)
U = copy.copy(U)

weights = copy.copy(weights)
W0.shape

(307, 4)

In [160]:
# Calcualte preconditioning coefficients (Step 2.2)

n_i = 0


p_i = 1.0 / 


p = []
for i in range(m):
    _p = 1 / (np.sum(weights[i] * W0[i]**2))
    p.append(_p)
p = np.array(p)
p.shape

ValueError: operands could not be broadcast together with shapes (41,) (4,) 

In [140]:
# Compute SE factors (Step 3.1)


# Compute the current fit (Step 3.2)
y_m = np.matmul(W0, H0)
y_m

array([[  0.53744906,   1.1851981 ,   1.2112601 , ...,   1.324988  ,
          6.3466654 ,   5.512077  ],
       [  0.3275335 ,  -0.10779054,  -0.6956849 , ...,  -0.50297207,
          5.1563826 ,   1.0174291 ],
       [ -0.9430924 ,  -2.170167  ,  -1.6131871 , ...,  -1.6596838 ,
         -4.3267317 , -13.820197  ],
       ...,
       [ -0.8138578 ,  -0.6944158 ,  -0.08305673, ...,  -0.16081484,
         -3.2473483 ,  -7.4857893 ],
       [ -1.0681205 ,  -2.263022  ,  -0.11232104, ...,  -0.61625844,
        -14.930397  , -15.483709  ],
       [  1.2652775 ,   0.96630913,  -0.18461367, ...,   0.32787272,
         17.25121   ,   7.7065096 ]], dtype=float32)

In [156]:
# Compute the gradient (Step 3.4)
residuals = X - y_m
i = 0

Wi = np.diagflat(weights[i])
wr = np.matmul(Wi, residuals.T)
wr.shape
g = np.matmul(H0, wr).T
g.shape

(307, 4)

In [ ]:
# Compute transofrmed gradient z (Step 3.5)
z = 

beta0 = 0
rho = np.matmul(g.T, 